In [3]:
!pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [37]:
df = df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/cluster_number_data.xlsx")
# 명사구 키워드(홍주)를 합치고 CLUSTER 한 데이터 셋을 사용
# tf-idf(희재) 부분 키워드 새로 추출하기 -- unknown이랑 foreign 추가해서
# 홍주가 뽑은 키워드에도 조사 같은 거는 할 것
# title_p와 content_p는 형식적으로 들어가 있는 단어들 제외하고 위 화살표같이 의미있는 특수기호는 문자로 바꾼 열임
# 데이터셋을 깔끔하게 하기 위해서 열을 정리
df1 = df.copy()
df1 = df[['date','title_p','content_p','len','collocation']]
df1

,date,title_p,content_p,len,collocation
0,2023-07-13 06:01:30,"차라리 자진 말소하겠다 임대인들, 보증보험 요건 강화 명단 공개에 반발","국토부, 7월 중 임대보증금 보증보험 가입요건 126퍼센트 강화 민간임대 특별법 시...",2081,"['2년 연속', '특별법 시행', '민간 임대주택', '126 퍼센트', '보증 ..."
1,2023-07-13 06:01:32,단맛의 세계 칼로리 안 낮은데 믿었던 제로 마케팅의 배신,작년 9월 출시 후 선풍적인 인기를 끈 롯데칠성음료의 처음처럼 새로 이하 새로 는 ...,1901,['제로 슈거']
2,2023-07-13 06:01:33,전세 사기보다 치솟은 월세가 더 무섭다 월세 수요 한풀 꺾여,6월 기준 서울 전세비중 51퍼센트대 회복 월세 급등에 전세로 선회 전세대출 금리 ...,1850,"['지난 5월', '임대 수요', '서울 아파트', '퍼센트 대로', '대출 규제'..."
3,2023-07-13 06:02:01,마켓PRO 칼럼 셈법 복잡해진 한국은행 금통위,한경 마켓PRO 텔레그램을 구독하시면 프리미엄 투자 콘텐츠를 보다 편리하게 볼 수 ...,1802,"['마켓 PRO', '소비자 물가', '금리 인상']"
4,2023-07-13 06:02:01,일주일 만에 두 배 뛴 금양 시장서는 이게 맞아,8거래일간 100퍼센트 넘게 뛰다 12일 10퍼센트대 급락 시총 6조 근접 한국항공...,1645,['한국 거래소']
...,...,...,...,...,...
1568,2023-07-14 05:41:58,"뉴욕증시, 인플레 완화에 상승 S&P500 4,500 돌파 마감",뉴욕 연합뉴스 윤영숙 연합인포맥스 특파원 뉴욕증시는 소비자 물가에 이어 생산자 물가...,2442,"['보험 청구', '실업 보험', '연방 준비', '생산자 물가', '전장 보다',..."
1569,2023-07-14 05:43:00,"손보사 5.3조 더벌고, 생보사 7.2조 덜번다 보험권 희비교차",올해 손해보험사와 생명보험사의 실적 희비가 엇갈릴 것이라는 보험연구원의 전망이 나왔...,1358,"['IFRS 17', '수입 보험료', '보험 연구원', '손해 보험사', '손해 ..."
1570,2023-07-14 05:46:01,"어쩌다 이렇게까지 한샘, 대표 교체로 쇄신 노린다",지난해 상장 이후 첫 적자 수익성 개선 노력했지만 성과 없어 IMM 경영에 적극 개...,1886,"['오퍼 레이', '실적 개선', '기업 가치', '신임 대표', 'IMM PE']"
1571,2023-07-14 05:53:01,"뉴욕증시, 대형기술주 강세 속에 나흘 연속 상승 엔비디아 4.7퍼센트",파이낸셜뉴스 뉴욕증시가 13일 이하 현지시간 대형 기술주 강세 속에 상승세를 이어갔...,1941,"['미래 수익', '예상 보다', '대형 기술', '물가 지수', '금리 인상', ..."


In [38]:
# TF-IDF 사용을 위한 라이브러리 설치 및 준비
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer()

In [39]:
# 가장 먼저 tf-idf를 사용해서 키워드 추출 작업 진행
df1['TF-keywords'] = None # tf-idf를 사용해서 뽑은 키워드를 저장할 새로운 열 생성

<ipython-input-39-7c9edc3f9a06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['TF-keywords'] = None # tf-idf를 사용해서 뽑은 키워드를 저장할 새로운 열 생성


In [40]:
# TF-IDF를 사영하여 문서 벡터화 진행
tfidf_matrix = tfidfv.fit_transform(df1['content_p'])

for i in range(len(df1)):
    doc_tfidf = tfidf_matrix[i].toarray().flatten() # 벡터화시킨 것을 다시 문자로
    sorted_indices = doc_tfidf.argsort()[::-1] # tf-idf값이 높을수록 중요한 키워드인데 높은 순으로 정렬
    top_keywords = [tfidfv.get_feature_names_out()[idx] for idx in sorted_indices[:15]]  # 상위 15개 단어만 추출 # 숫자나 영어 단어만 나올 경우 제거하고 다음 키워드를 올려서 쓸거기 때문에 넉넉히 뽑음
    # 키워드를 저장할 빈 리스트 생성
    keywords = []
    keywords.append(top_keywords)
    # 각 행에 알맞은 키워드 부여
    df1['TF-keywords'][i] = keywords
print(df1['TF-keywords'][:10])

<ipython-input-40-2cedfae31f9f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['TF-keywords'][i] = keywords


0    [[임대보증금, 공시가격의, 임대인, 특별법, 보증금, 임대인들이, 민간임대주택에,...
1    [[제로, 열량은, 말티톨은, 발암, 설탕, 아스파탐, 다이어트에, 설탕이, 열량,...
2    [[월세, 전세, 대출, 아파트, 전세대출, 전세로, 수요가, 임대, 주택, 서울,...
3    [[듯하다, 소비자물가는, 안으로는, 금리인상을, 마켓pro, 여겨진다, 고용과, ...
4    [[2차전지, 금양의, 공매도, 금양, 주가, 원을, 금양에, 발포제, 배로, 한국...
5    [[무상증자를, 1주당, 유상증자를, 보통주, 기업들이, 투자자들에게, 탓이다, 주...
6    [[p500의, 브로드컴은, arm, cpi, 지수는, 최저를, 로이터에, 경쟁사,...
7    [[하루인베스트, 가상자산, 하루인베스트는, 출금, 피해자들은, 박씨는, 자산, 이...
8    [[연준은, 사정권에, 근원, 주택, 인플레이션이, 불구하고, 기대했다, cpi보다...
9    [[가계대출, 은행권, 증가, 주담대, 기타대출, 입주물량, 전월, 가계대출도, 잔...
Name: TF-keywords, dtype: object


In [41]:
df1.to_excel('TF-KEYWORDS 15개 8월2일 중간파일 저장.xlsx', index=False)  # index=False를 지정하여 인덱스를 파일에 포함하지 않습니다.

In [113]:
# 위 키워드 추출 작업이 한시간 걸리기 때문에 중간 파일 저장 해두고 여기서부터 하면 됌 다음에 수정할 때는
df1 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TF-KEYWORDS 15개 8월2일 중간파일 저장.xlsx")
df1.head()

,date,title_p,content_p,len,collocation,TF-keywords
0,2023-07-13 06:01:30,"차라리 자진 말소하겠다 임대인들, 보증보험 요건 강화 명단 공개에 반발","국토부, 7월 중 임대보증금 보증보험 가입요건 126퍼센트 강화 민간임대 특별법 시...",2081,"['2년 연속', '특별법 시행', '민간 임대주택', '126 퍼센트', '보증 ...","[['임대보증금', '공시가격의', '임대인', '특별법', '보증금', '임대인들..."
1,2023-07-13 06:01:32,단맛의 세계 칼로리 안 낮은데 믿었던 제로 마케팅의 배신,작년 9월 출시 후 선풍적인 인기를 끈 롯데칠성음료의 처음처럼 새로 이하 새로 는 ...,1901,['제로 슈거'],"[['제로', '열량은', '말티톨은', '발암', '설탕', '아스파탐', '다이..."
2,2023-07-13 06:01:33,전세 사기보다 치솟은 월세가 더 무섭다 월세 수요 한풀 꺾여,6월 기준 서울 전세비중 51퍼센트대 회복 월세 급등에 전세로 선회 전세대출 금리 ...,1850,"['지난 5월', '임대 수요', '서울 아파트', '퍼센트 대로', '대출 규제'...","[['월세', '전세', '대출', '아파트', '전세대출', '전세로', '수요가..."
3,2023-07-13 06:02:01,마켓PRO 칼럼 셈법 복잡해진 한국은행 금통위,한경 마켓PRO 텔레그램을 구독하시면 프리미엄 투자 콘텐츠를 보다 편리하게 볼 수 ...,1802,"['마켓 PRO', '소비자 물가', '금리 인상']","[['듯하다', '소비자물가는', '안으로는', '금리인상을', '마켓pro', '..."
4,2023-07-13 06:02:01,일주일 만에 두 배 뛴 금양 시장서는 이게 맞아,8거래일간 100퍼센트 넘게 뛰다 12일 10퍼센트대 급락 시총 6조 근접 한국항공...,1645,['한국 거래소'],"[['2차전지', '금양의', '공매도', '금양', '주가', '원을', '금양에..."


In [114]:
df1['TF-keywords'][0]

"[['임대보증금', '공시가격의', '임대인', '특별법', '보증금', '임대인들이', '민간임대주택에', '보증보험', '집단', '시행령', '전세보증금', '자진말소를', '가입요건', '과태료', '임대인의']]"

In [115]:
type(df1['TF-keywords'][0])

str

In [116]:
# 키워드들이 리스트 속에 리스트로 저장되어 있기 대문에 리스트 하나를 언패킹해주는 작업이 필요함

#for i in range(len(df1['TF-keywords'])):
#  df1['TF-keywords'][i] = df1['TF-keywords'][i][0]

#df1['TF-keywords'][0]

In [ ]:
# 원래는 위 코드가 맞는데 엑셀에 저장되면서 리스트가 텍스트로 바뀌는 현상이 발생함

In [117]:
import json

for i in range(len(df1)):
    data_str = df1['TF-keywords'][i]
    data_str_fixed = data_str.replace("'", "\"")  # 작은 따옴표를 큰 따옴표로 변경
    data_list = json.loads(data_str_fixed)
    df1['TF-keywords'][i] = data_list


<ipython-input-117-27bacaf69ddd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['TF-keywords'][i] = data_list


In [118]:
# 다시 원래처럼 리스트속에 리스트가 존재하도록 바꾸어줌 EX) [['A','B','C']]
df1['TF-keywords'][0]

[['임대보증금',
  '공시가격의',
  '임대인',
  '특별법',
  '보증금',
  '임대인들이',
  '민간임대주택에',
  '보증보험',
  '집단',
  '시행령',
  '전세보증금',
  '자진말소를',
  '가입요건',
  '과태료',
  '임대인의']]

In [119]:
for i in range(len(df1['TF-keywords'])):
  df1['TF-keywords'][i] = df1['TF-keywords'][i][0]

df1['TF-keywords'][0]

<ipython-input-119-e8b2f8c1af89>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['TF-keywords'][i] = df1['TF-keywords'][i][0]


['임대보증금',
 '공시가격의',
 '임대인',
 '특별법',
 '보증금',
 '임대인들이',
 '민간임대주택에',
 '보증보험',
 '집단',
 '시행령',
 '전세보증금',
 '자진말소를',
 '가입요건',
 '과태료',
 '임대인의']

In [120]:
# TF-IDF를 사용하여 키워드를 잡은 후 추출된 키워들에 붙은 을/를 특수기호 같은 것들을 제거해줌
# 그리고 홍주 키워드들도 같은 작업 진행할 것

# 조사를 구별해내기 위해서 형태소 분석기 okt 사용
# okt 형태소 분석기를 위한 라이브러리 설치 및 준비
!pip install konlpy
from konlpy.tag import Okt

In [121]:
# 형태소 분석 결과 확인
for i in range(10):
  for j in range(len(df1['TF-keywords'][i])):
    print(okt.pos(df1['TF-keywords'][i][j]))

[('임대', 'Noun'), ('보증금', 'Noun')]
[('공시가격', 'Noun'), ('의', 'Josa')]
[('임대', 'Noun'), ('인', 'Josa')]
[('특별법', 'Noun')]
[('보증금', 'Noun')]
[('임대', 'Noun'), ('인들', 'Josa'), ('이', 'Determiner')]
[('민간', 'Noun'), ('임대주택', 'Noun'), ('에', 'Josa')]
[('보증', 'Noun'), ('보험', 'Noun')]
[('집단', 'Noun')]
[('시행', 'Noun'), ('령', 'Noun')]
[('전세', 'Noun'), ('보증금', 'Noun')]
[('자진말', 'Verb'), ('소', 'Noun'), ('를', 'Josa')]
[('가입', 'Noun'), ('요건', 'Noun')]
[('과태료', 'Noun')]
[('임대', 'Noun'), ('인', 'Suffix'), ('의', 'Josa')]
[('제로', 'Noun')]
[('열량', 'Noun'), ('은', 'Josa')]
[('말티톨', 'Noun'), ('은', 'Josa')]
[('발암', 'Noun')]
[('설탕', 'Noun')]
[('아스파탐', 'Noun')]
[('다이어트', 'Noun'), ('에', 'Josa')]
[('설탕', 'Noun'), ('이', 'Josa')]
[('열량', 'Noun')]
[('진로', 'Noun')]
[('혼란', 'Noun'), ('스러워하고', 'Adjective')]
[('도수', 'Noun'), ('를', 'Josa')]
[('설탕', 'Noun'), ('의', 'Josa')]
[('식품', 'Noun'), ('회사', 'Noun'), ('들', 'Suffix'), ('이', 'Josa')]
[('제로', 'Noun'), ('슈거', 'Noun')]
[('월세', 'Noun')]
[('전세', 'Noun')]
[('대출', 'Noun')]
[('아파트'

In [122]:
# 형태소 분석기 지정
okt = Okt()

In [123]:
# 불용어 사전 파일을 읽어와서 불용어 사전 구성
# 배포되어 있는 불용어 사전에 개인작업으로 추가 불용어들 추가 >> 은정이 파일
with open('/content/drive/MyDrive/Colab Notebooks/new_stopwords.txt', 'r', encoding='utf-8') as f:
    korean_stopwords = set(f.read().splitlines())

# 형태소 분석 후 명사와 숫자, 알파벳, 접미사만 남기는 함수 정의
def filter_words(text):
    morphemes = okt.pos(text)
    filtered_words = [word for word, pos in morphemes if pos in ['Noun', 'Number', 'Alpha', 'Suffix','Foreign'] and word not in korean_stopwords and pos !='Josa']
    return ' '.join(filtered_words)
# 'FOREIGN'은 추가했는데 unknown은 없어서 추가 안했음!

# 명사 숫자 알파벳 등등만 남긴 새로운 키워드 저장할 새로운 열 생성
df1['New_keys'] = None
# 위 에 업데이트 해주기
for i in range(len(df1['TF-keywords'])):
    new_keywords_list = []
    for j in range(len(df1['TF-keywords'][i])):
        filtered_words = filter_words(df1['TF-keywords'][i][j])
        new_keywords_list.append(filtered_words)
    df1.at[i, 'New_keys'] = new_keywords_list

print(df1['New_keys'])

0       [임대 보증금, 공시가격, 임대, 특별법, 보증금, 임대, 민간 임대주택, 보증 보...
1       [제로, 열량, 말티톨, 발암, 설탕, 아스파탐, 다이어트, 설탕, 열량, 진로, ...
2       [월세, 전세, 대출, 아파트, 전세 대출, 전세, 수요, 임대, 주택, 서울, 가...
3       [듯, 소비자 물가, , 금리 인상, 마켓 pro, , 고용, 가운데, 방향성, 입...
4       [2, 금 양, 공매도, 금 양, 주가, 원, 금 양, 발포, 배, 한국 항공우주,...
                              ...                        
1568    [준, 주간, 전장, 금리, 불러드, 투자자, 전달, 청구, 실업 보험, , 생산자...
1569    [수입 보험료, 보험사, 보험사, csm, csm 은, 보험, 생명 보험사, 보험 ...
1570    [imm, pe, 샘, 신임, 김유진, 경영, 대표, 개선, 적자, 김진태, 샘, ...
1571    [아마존, 금리 인상, 일비, 로웬 가트, 54달러, 상승, , 전날, 프라임 데이...
1572    [가계부채, 총재, gdp, 가계부채, 부동산, 배경, 문제, 가계 대출, 금리, ...
Name: New_keys, Length: 1573, dtype: object


In [124]:
df1['New_keys'][0]

['임대 보증금',
 '공시가격',
 '임대',
 '특별법',
 '보증금',
 '임대',
 '민간 임대주택',
 '보증 보험',
 '집단',
 '시행',
 '전세 보증금',
 '소',
 '가입 요건',
 '과태료',
 '임대']

In [125]:
# 홍주 키워드도 엑셀로 저장하면서 데이터 형태가 바뀜
df1['collocation'][0][0]

'['

In [126]:
import ast
# 데이터프레임에 있는 'collocation' 열의 모든 데이터를 리스트로 변환
df1['collocation'] = df1['collocation'].apply(ast.literal_eval)

# 변환된 'collocation' 열 출력
print(df1['collocation'][0][0])

2년 연속


In [127]:
# 형태소 분석 결과 확인
for i in range(10):
  for j in range(len(df1['collocation'][i])):
    print(okt.pos(df1['collocation'][i][j]))

[('2년', 'Number'), ('연속', 'Noun')]
[('특별법', 'Noun'), ('시행', 'Noun')]
[('민간', 'Noun'), ('임대주택', 'Noun')]
[('126', 'Number'), ('퍼센트', 'Noun')]
[('보증', 'Noun'), ('보험', 'Noun')]
[('등록', 'Noun'), ('말소', 'Verb')]
[('보증금', 'Noun'), ('반환', 'Noun')]
[('보험', 'Noun'), ('가입', 'Noun')]
[('전세', 'Noun'), ('보증금', 'Noun'), ('반환', 'Noun')]
[('보증', 'Noun'), ('보험', 'Noun'), ('가입', 'Noun')]
[('보증금', 'Noun'), ('반환', 'Noun'), ('보증', 'Noun')]
[('보증금', 'Noun'), ('보증', 'Noun'), ('보험', 'Noun')]
[('임대', 'Noun'), ('보증금', 'Noun'), ('보증', 'Noun')]
[('임대', 'Noun'), ('보증금', 'Noun'), ('보증', 'Noun'), ('보험', 'Noun')]
[('가입', 'Noun'), ('요건', 'Noun')]
[('임대', 'Noun'), ('사업자', 'Noun')]
[('제로', 'Noun'), ('슈거', 'Noun')]
[('지난', 'Noun'), ('5월', 'Number')]
[('임대', 'Noun'), ('수요', 'Noun')]
[('서울', 'Noun'), ('아파트', 'Noun')]
[('퍼센트', 'Noun'), ('대로', 'Noun')]
[('대출', 'Noun'), ('규제', 'Noun')]
[('대출', 'Noun'), ('금리', 'Noun')]
[('평균', 'Noun'), ('월세', 'Noun')]
[('월세', 'Noun'), ('가격', 'Noun')]
[('아파트', 'Noun'), ('월세', 'Noun')]
[('전세', '

In [128]:
# 위에서 형태소분석해본 결과 정상적으로 작동함
# 이제 조사나 그런거 없애는 작업을 똑같이 홍주 키워드에도 똑같은 작업 진행

# 명사 숫자 알파벳 등등만 남긴 새로운 키워드 저장할 새로운 열 생성
df1['New_collocation'] = None
# 위 에 업데이트 해주기
for i in range(len(df1['collocation'])):
    new_keywords_list = []
    for j in range(len(df1['collocation'][i])):
        filtered_words = filter_words(df1['collocation'][i][j])
        new_keywords_list.append(filtered_words)
    df1.at[i, 'New_collocation'] = new_keywords_list

print(df1['New_collocation'])

0       [2년 연속, 특별법 시행, 민간 임대주택, 126 퍼센트, 보증 보험, 등록, 보...
1                                                 [제로 슈거]
2       [지난 5월, 임대 수요, 서울 아파트, 퍼센트, 대출 규제, 대출 금리, 평균 월...
3                                 [마켓 PRO, 소비자 물가, 금리 인상]
4                                                [한국 거래소]
                              ...                        
1568    [보험 청구, 실업 보험, 연방 준비, 생산자 물가, 전장, 500 지수, 예상, ...
1569             [IFRS 17, 수입 보험료, 보험 연구원, 손해 보험사, 손해 보험]
1570                    [레이, 실적 개선, 기업 가치, 신임 대표, IMM PE]
1571    [미래 수익, 예상, 대형 기술, 물가 지수, 금리 인상, 시장 예상, 퍼센트 급등...
1572    [배경 설명, 부동산 시장, 가계 대출, GDP, 대출 증가, 기적, 가계부채 비율...
Name: New_collocation, Length: 1573, dtype: object


In [129]:
# collocation이랑 tf-keywords열은 이제 사용안해서 빼주기
df2 = df1[['date','title_p','content_p','len','New_keys','New_collocation']]
df2.head()

,date,title_p,content_p,len,New_keys,New_collocation
0,2023-07-13 06:01:30,"차라리 자진 말소하겠다 임대인들, 보증보험 요건 강화 명단 공개에 반발","국토부, 7월 중 임대보증금 보증보험 가입요건 126퍼센트 강화 민간임대 특별법 시...",2081,"[임대 보증금, 공시가격, 임대, 특별법, 보증금, 임대, 민간 임대주택, 보증 보...","[2년 연속, 특별법 시행, 민간 임대주택, 126 퍼센트, 보증 보험, 등록, 보..."
1,2023-07-13 06:01:32,단맛의 세계 칼로리 안 낮은데 믿었던 제로 마케팅의 배신,작년 9월 출시 후 선풍적인 인기를 끈 롯데칠성음료의 처음처럼 새로 이하 새로 는 ...,1901,"[제로, 열량, 말티톨, 발암, 설탕, 아스파탐, 다이어트, 설탕, 열량, 진로, ...",[제로 슈거]
2,2023-07-13 06:01:33,전세 사기보다 치솟은 월세가 더 무섭다 월세 수요 한풀 꺾여,6월 기준 서울 전세비중 51퍼센트대 회복 월세 급등에 전세로 선회 전세대출 금리 ...,1850,"[월세, 전세, 대출, 아파트, 전세 대출, 전세, 수요, 임대, 주택, 서울, 가...","[지난 5월, 임대 수요, 서울 아파트, 퍼센트, 대출 규제, 대출 금리, 평균 월..."
3,2023-07-13 06:02:01,마켓PRO 칼럼 셈법 복잡해진 한국은행 금통위,한경 마켓PRO 텔레그램을 구독하시면 프리미엄 투자 콘텐츠를 보다 편리하게 볼 수 ...,1802,"[듯, 소비자 물가, , 금리 인상, 마켓 pro, , 고용, 가운데, 방향성, 입...","[마켓 PRO, 소비자 물가, 금리 인상]"
4,2023-07-13 06:02:01,일주일 만에 두 배 뛴 금양 시장서는 이게 맞아,8거래일간 100퍼센트 넘게 뛰다 12일 10퍼센트대 급락 시총 6조 근접 한국항공...,1645,"[2, 금 양, 공매도, 금 양, 주가, 원, 금 양, 발포, 배, 한국 항공우주,...",[한국 거래소]


In [130]:
# 띄어쓰기를 없애는 함수 정의
def remove_whitespace(text_list):
    return [text.replace(" ", "") for text in text_list]

# df2['keywords'], df2['collocation'] 에 적용하여 띄어쓰기 없애기
df2['New_keys'] = df2['New_keys'].apply(remove_whitespace)
df2['New_collocation'] = df2['New_collocation'].apply(remove_whitespace)


# 각 행에 있는 리스트에서 " "이거나 비어있는 인덱스를 제외하는 함수 정의
def remove_empty_and_whitespace_from_list(text_list):
    # 단어가 1개 이상의 문자로 이루어진 경우에만 유지하도록 조건 추가
    return [text for text in text_list if text.strip() != "" and len(text) > 1]

# df1['keywords'], df1['collocation에 적용하여 " "이거나 비어있는 인덱스와 하나의 문자로만 이루어진 단어 제외
df2['New_keys'] = df2['New_keys'].apply(remove_empty_and_whitespace_from_list)
df2['New_collocation'] = df2['New_collocation'].apply(remove_empty_and_whitespace_from_list)

# 결과 출력
print(df2['New_keys'])
print(df2['New_collocation'])

0       [임대보증금, 공시가격, 임대, 특별법, 보증금, 임대, 민간임대주택, 보증보험, ...
1       [제로, 열량, 말티톨, 발암, 설탕, 아스파탐, 다이어트, 설탕, 열량, 진로, ...
2       [월세, 전세, 대출, 아파트, 전세대출, 전세, 수요, 임대, 주택, 서울, 가격...
3       [소비자물가, 금리인상, 마켓pro, 고용, 가운데, 방향성, 입장, 25bp, 비...
4       [금양, 공매도, 금양, 주가, 금양, 발포, 한국항공우주, 금양은, 원통형, 급등...
                              ...                        
1568    [주간, 전장, 금리, 불러드, 투자자, 전달, 청구, 실업보험, 생산자, 기업, ...
1569    [수입보험료, 보험사, 보험사, csm, csm은, 보험, 생명보험사, 보험연구원,...
1570    [imm, pe, 신임, 김유진, 경영, 대표, 개선, 적자, 김진태, 부지, pe...
1571    [아마존, 금리인상, 일비, 로웬가트, 54달러, 상승, 전날, 프라임데이, 시장,...
1572    [가계부채, 총재, gdp, 가계부채, 부동산, 배경, 문제, 가계대출, 금리, 총...
Name: New_keys, Length: 1573, dtype: object
0       [2년연속, 특별법시행, 민간임대주택, 126퍼센트, 보증보험, 등록, 보증금반환,...
1                                                  [제로슈거]
2       [지난5월, 임대수요, 서울아파트, 퍼센트, 대출규제, 대출금리, 평균월세, 월세가...
3                                    [마켓PRO, 소비자물가, 금리인상]
4                                                 [한국거래소]
                            

<ipython-input-130-d555ba353197>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['New_keys'] = df2['New_keys'].apply(remove_whitespace)
<ipython-input-130-d555ba353197>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['New_collocation'] = df2['New_collocation'].apply(remove_whitespace)
<ipython-input-130-d555ba353197>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [131]:
# 중복제거 해주기
def select_unique_words(text_list):
    unique_words = list(set(text_list))
    return unique_words

df2['New_keys'] = df2['New_keys'].apply(select_unique_words)
print(df2['New_keys'])

df2['New_collocation'] = df2['New_collocation'].apply(select_unique_words)
print(df2['New_collocation'])


0       [민간임대주택, 집단, 특별법, 임대, 임대보증금, 가입요건, 공시가격, 보증보험,...
1       [말티톨, 제로, 제로슈거, 혼란, 열량, 식품회사, 발암, 도수, 다이어트, 설탕...
2       [수요, 월세, 주택, 아파트, 임대, 서울, 대출, 전세대출, 립다세대, 선회, ...
3       [25bp, 마켓pro, 소비자물가, 방향성, 입장, 금리인상, 비교적, 물가, 가...
4       [금양은, 발포, 한국항공우주, 급등, 공매도, 원통형, 6조, 금양, 한국거래소,...
                              ...                        
1568    [청구, 10bp, 금리, 실업보험, 투자자, 기업, 불러드, 1주일, 시즌, 생산...
1569    [보험연구원, 손해보험사, 보험사, 이익, 생명보험사, csm, 수입보험료, csm...
1570    [개선, 지난해, 김진태, 부지, 적자, 기업, 김유진, pe, 신임, 대표, 경영...
1571    [전날, 시장, 일비, 아마존, 알파벳, 상승, 미래, 금리인상, 54달러, 로웬가...
1572    [금리, 부동산, 대응, 은행, 총재, gdp, 가계부채, 설명, 문제, 배경, 가...
Name: New_keys, Length: 1573, dtype: object
0       [민간임대주택, 등록, 보험가입, 임대보증금보증, 특별법시행, 임대보증금보증보험, ...
1                                                  [제로슈거]
2       [월세가격, 퍼센트, 전세대출금리, 서울아파트, 대출규제, 지난5월, 평균월세, 임...
3                                    [금리인상, 소비자물가, 마켓PRO]
4                                                 [한국거래소]
                            

<ipython-input-131-886fcc47cf15>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['New_keys'] = df2['New_keys'].apply(select_unique_words)
<ipython-input-131-886fcc47cf15>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['New_collocation'] = df2['New_collocation'].apply(select_unique_words)


In [132]:
df2

,date,title_p,content_p,len,New_keys,New_collocation
0,2023-07-13 06:01:30,"차라리 자진 말소하겠다 임대인들, 보증보험 요건 강화 명단 공개에 반발","국토부, 7월 중 임대보증금 보증보험 가입요건 126퍼센트 강화 민간임대 특별법 시...",2081,"[민간임대주택, 집단, 특별법, 임대, 임대보증금, 가입요건, 공시가격, 보증보험,...","[민간임대주택, 등록, 보험가입, 임대보증금보증, 특별법시행, 임대보증금보증보험, ..."
1,2023-07-13 06:01:32,단맛의 세계 칼로리 안 낮은데 믿었던 제로 마케팅의 배신,작년 9월 출시 후 선풍적인 인기를 끈 롯데칠성음료의 처음처럼 새로 이하 새로 는 ...,1901,"[말티톨, 제로, 제로슈거, 혼란, 열량, 식품회사, 발암, 도수, 다이어트, 설탕...",[제로슈거]
2,2023-07-13 06:01:33,전세 사기보다 치솟은 월세가 더 무섭다 월세 수요 한풀 꺾여,6월 기준 서울 전세비중 51퍼센트대 회복 월세 급등에 전세로 선회 전세대출 금리 ...,1850,"[수요, 월세, 주택, 아파트, 임대, 서울, 대출, 전세대출, 립다세대, 선회, ...","[월세가격, 퍼센트, 전세대출금리, 서울아파트, 대출규제, 지난5월, 평균월세, 임..."
3,2023-07-13 06:02:01,마켓PRO 칼럼 셈법 복잡해진 한국은행 금통위,한경 마켓PRO 텔레그램을 구독하시면 프리미엄 투자 콘텐츠를 보다 편리하게 볼 수 ...,1802,"[25bp, 마켓pro, 소비자물가, 방향성, 입장, 금리인상, 비교적, 물가, 가...","[금리인상, 소비자물가, 마켓PRO]"
4,2023-07-13 06:02:01,일주일 만에 두 배 뛴 금양 시장서는 이게 맞아,8거래일간 100퍼센트 넘게 뛰다 12일 10퍼센트대 급락 시총 6조 근접 한국항공...,1645,"[금양은, 발포, 한국항공우주, 급등, 공매도, 원통형, 6조, 금양, 한국거래소,...",[한국거래소]
...,...,...,...,...,...,...
1568,2023-07-14 05:41:58,"뉴욕증시, 인플레 완화에 상승 S&P500 4,500 돌파 마감",뉴욕 연합뉴스 윤영숙 연합인포맥스 특파원 뉴욕증시는 소비자 물가에 이어 생산자 물가...,2442,"[청구, 10bp, 금리, 실업보험, 투자자, 기업, 불러드, 1주일, 시즌, 생산...","[실업보험, 보험청구, 퍼센트상승, 500지수, 생산자물가, 예상, 국채금리, 연방..."
1569,2023-07-14 05:43:00,"손보사 5.3조 더벌고, 생보사 7.2조 덜번다 보험권 희비교차",올해 손해보험사와 생명보험사의 실적 희비가 엇갈릴 것이라는 보험연구원의 전망이 나왔...,1358,"[보험연구원, 손해보험사, 보험사, 이익, 생명보험사, csm, 수입보험료, csm...","[손해보험사, 보험연구원, IFRS17, 손해보험, 수입보험료]"
1570,2023-07-14 05:46:01,"어쩌다 이렇게까지 한샘, 대표 교체로 쇄신 노린다",지난해 상장 이후 첫 적자 수익성 개선 노력했지만 성과 없어 IMM 경영에 적극 개...,1886,"[개선, 지난해, 김진태, 부지, 적자, 기업, 김유진, pe, 신임, 대표, 경영...","[신임대표, 실적개선, 레이, 기업가치, IMMPE]"
1571,2023-07-14 05:53:01,"뉴욕증시, 대형기술주 강세 속에 나흘 연속 상승 엔비디아 4.7퍼센트",파이낸셜뉴스 뉴욕증시가 13일 이하 현지시간 대형 기술주 강세 속에 상승세를 이어갔...,1941,"[전날, 시장, 일비, 아마존, 알파벳, 상승, 미래, 금리인상, 54달러, 로웬가...","[퍼센트오른, 퍼센트상승, 물가지수, 예상, 금리인상, 시장예상, 퍼센트급등, 미래..."


In [133]:
print(type(df2['New_collocation'][0]))
print(type(df2['New_keys'][0]))

<class 'list'>
<class 'list'>


In [134]:
# 두 열을 합쳐서 저장할 새로운 열 생성
df2['merged_col'] = None

<ipython-input-134-3f159ef1210c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['merged_col'] = None


In [135]:
# 새로 만든 열에 합쳐서 저장
df2['merged_col'] = df2.apply(lambda row: row['New_keys'] + row['New_collocation'], axis=1)

<ipython-input-135-09ef5afe1dc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['merged_col'] = df2.apply(lambda row: row['New_keys'] + row['New_collocation'], axis=1)


In [136]:
# 중복을 제거하기 위해서 set 사용
df2['merged_col'] = df2['merged_col'].apply(lambda x: list(set(x)))

<ipython-input-136-b510f318d5c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['merged_col'] = df2['merged_col'].apply(lambda x: list(set(x)))


In [137]:
df2['merged_col'].head()

0    [특별법시행, 가입요건, 보증보험가입, 전세보증금, 등록, 보험가입, 집단, 임대,...
1    [말티톨, 제로, 제로슈거, 혼란, 열량, 식품회사, 발암, 도수, 다이어트, 설탕...
2    [수요, 서울, 대출규제, 평균월세, 립다세대, 임대수요, 대출금리, 아파트, 임대...
3    [가운데, 25bp, 마켓pro, 소비자물가, 입장, 방향성, 금리인상, 마켓PRO...
4    [금양은, 발포, 한국항공우주, 급등, 공매도, 원통형, 6조, 금양, 한국거래소,...
Name: merged_col, dtype: object

In [138]:
df2.to_excel('TF-KEYWORDS 15개 + 홍주 키워드 형태소 분석 8월2일.xlsx', index=False)  # index=False를 지정하여 인덱스를 파일에 포함하지 않습니다.